# Building Relation Network Using Tensorflow

Relation network is pretty simple, right? Now, we will understand relation network better by implementing it in TensorFlow. We will consider a simple binary classification problem and see how can we solve them using a relation network. 

First, we import all the required libraries,

In [1]:
import tensorflow as tf
import numpy as np

Let us say we have two classes in our dataset, we will randomly generate some 1000 data points for each of these classes.

In [2]:
classA = np.random.rand(1000,18)

ClassB = np.random.rand(1000,18)

Create our dataset by combining both of these classes,

In [3]:
data = np.vstack([classA, ClassB])

Now we will set the label, we assign label 1 for classA and label 0 for classB. 

In [4]:
label = np.vstack([np.ones((len(classA),1)),np.zeros((len(ClassB),1))])

So, our dataset will have 2000 records

In [5]:
data.shape

(2000, 18)

In [6]:
label.shape

(2000, 1)

Now, we will define the placeholders for our support set $x_i$ and query set $x_j$. 

In [7]:
xi = tf.placeholder(tf.float32, [None, 9])
xj = tf.placeholder(tf.float32, [None, 9])

Define the placeholder for label y,

In [8]:
y = tf.placeholder(tf.float32, [None, 1]) 

Now, we will define our embedding function $f_{\varphi}$()  to learn the embeddings of support set and query set. We will use a normal feedforward network as our embedding function. 

In [9]:
def embedding_function(x):
    
    weights = tf.Variable(tf.truncated_normal([9,1]))
    bias = tf.Variable(tf.truncated_normal([1]))
    
    a = (tf.nn.xw_plus_b(x,weights,bias))
    embeddings = tf.nn.relu(a)
    
    return embeddings

Compute the embeddings of our support set i.e $f_{\varphi}(x_i) $

In [10]:
f_xi = embedding_function(xi)

Compute the embeddings of our query set i.e $f_{\varphi}(x_j)$

In [11]:
f_xj = embedding_function(xj)

Now that we calculated the embeddings and have the feature vectors, we combine both the support set and query set feature vectors i.e $Z(f_{\varphi}(x_i), f_{\varphi}(x_j))$

In [12]:
Z = tf.concat([f_xi,f_xj],axis=1)

We define our relation function $g_{\phi}()$ as three layered neural network with relu activations, 

In [13]:
def relation_function(x):
    w1 = tf.Variable(tf.truncated_normal([2,3]))
    b1 = tf.Variable(tf.truncated_normal([3]))
    
    w2 = tf.Variable(tf.truncated_normal([3,5]))
    b2 = tf.Variable(tf.truncated_normal([5]))
    
    w3 = tf.Variable(tf.truncated_normal([5,1]))
    b3 = tf.Variable(tf.truncated_normal([1]))
    
    #layer1
    z1 = (tf.nn.xw_plus_b(x,w1,b1))
    a1 = tf.nn.relu(z1)
    
    #layer2
    z2 = tf.nn.xw_plus_b(a1,w2,b2)
    a2 = tf.nn.relu(z2)
    
    #layer3
    z3 = tf.nn.xw_plus_b(z2,w3,b3)

    #output
    y = tf.nn.sigmoid(z3)
    
    return y

We now pass the concatenated feature vectors of support set and query set to the relation function and get the relation scores, 

In [14]:
relation_scores = relation_function(Z)

We define our loss function as mean squared error i.e squared difference between relation scores and actual y value. 

In [15]:
loss_function = tf.reduce_mean(tf.squared_difference(relation_scores,y))

We can minimize the loss using adam optimizer,

In [16]:
optimizer = tf.train.AdamOptimizer(0.1)
train = optimizer.minimize(loss_function)

Now, let's start our tensorflow session

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

Now, we randomly sample data points for our support set $x_i$ and query set $x_j$ and train the network, 

In [ ]:
for episode in range(1000):
    _, loss_value = sess.run([train, loss_function], 
                             feed_dict={xi:data[:,0:9]+np.random.randn(*np.shape(data[:,0:9]))*0.05,
                                        xj:data[:,9:]+np.random.randn(*np.shape(data[:,9:]))*0.05,
                                        y:label})
    if episode % 100 == 0:
        print("Episode {}: loss {:.3f} ".format(episode, loss_value))

Episode 0: loss 0.400 
Episode 100: loss 0.250 
Episode 200: loss 0.250 
Episode 300: loss 0.250 
Episode 400: loss 0.250 
Episode 500: loss 0.250 
Episode 600: loss 0.250 
